In [1]:
# Ячейка 1: Установка зависимостей 
!pip install selenium requests


In [2]:
# Ячейка 2: Импорт необходимых модулей и определение Page Object

import unittest
import time
import requests
import random
import string
import sys
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoAlertPresentException, TimeoutException

# Page Object для страницы книг
class BooksPage:
    def __init__(self, driver):
        self.driver = driver

    def load(self):
        """Открывает страницу https://demoqa.com/books и ждёт появления поискового поля."""
        self.driver.get("https://demoqa.com/books")
        WebDriverWait(self.driver, 10).until(
            EC.presence_of_element_located((By.ID, "searchBox"))
        )
    
    def search_book(self, query):
        """Вводит текст запроса в поле поиска и ждёт обновления результатов."""
        search_input = self.driver.find_element(By.ID, "searchBox")
        search_input.clear()
        search_input.send_keys(query)
        time.sleep(1)  # ожидание обновления таблицы
    
    def get_book_rows(self):
        """Возвращает список строк (элементов) таблицы с книгами."""
        return self.driver.find_elements(By.XPATH, "//div[@class='rt-tbody']/div")
    
    def click_book(self, book_title):
        """Находит строку, содержащую указанный заголовок книги, и кликает по ней."""
        rows = self.get_book_rows()
        for row in rows:
            if book_title.lower() in row.text.lower():
                row.click()
                return True
        return False
    
    def get_table_headers(self):
        """Возвращает список заголовков таблицы."""
        return [header.text for header in self.driver.find_elements(By.XPATH, "//div[@class='rt-thead -header']/div/div")]

# Page Object для страницы входа (Login)
class LoginPage:
    def __init__(self, driver):
        self.driver = driver

    def load(self):
        self.driver.get("https://demoqa.com/login")
        WebDriverWait(self.driver, 10).until(
            EC.presence_of_element_located((By.ID, "userName"))
        )
    
    def login(self, username, password):
        self.load()
        user_input = self.driver.find_element(By.ID, "userName")
        pass_input = self.driver.find_element(By.ID, "password")
        user_input.clear()
        user_input.send_keys(username)
        pass_input.clear()
        pass_input.send_keys(password)
        # Ждём, пока кнопка станет кликабельной, и прокручиваем её в видимую область
        login_btn = WebDriverWait(self.driver, 10).until(
            EC.element_to_be_clickable((By.ID, "login"))
        )
        self.driver.execute_script("arguments[0].scrollIntoView(true);", login_btn)
        login_btn.click()


# Page Object для страницы профиля (личного кабинета)
class ProfilePage:
    def __init__(self, driver):
        self.driver = driver

    def load(self):
        """Переходит на страницу профиля."""
        self.driver.get("https://demoqa.com/profile")
        WebDriverWait(self.driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//div[@class='rt-tbody']"))
        )
    
    def get_books_in_collection(self):
        """Возвращает список книг в коллекции пользователя."""
        return self.driver.find_elements(By.XPATH, "//div[@class='rt-tbody']/div")
    
    def remove_book(self, book_title):
        """Удаляет книгу из коллекции по названию."""
        rows = self.get_books_in_collection()
        for row in rows:
            if book_title.lower() in row.text.lower():
                # Предполагается, что в каждой строке есть кнопка "Delete"
                delete_btn = row.find_element(By.XPATH, ".//span[text()='Delete']")
                delete_btn.click()
                # Подтверждение удаления
                WebDriverWait(self.driver, 10).until(
                    EC.element_to_be_clickable((By.ID, "closeSmallModal-ok"))
                ).click()
                return True
        return False


In [3]:
# Ячейка 3: Тесты для главной страницы

class MainPageTests(unittest.TestCase):
    @classmethod
    def setUpClass(cls):
        options = webdriver.ChromeOptions()
        options.add_argument("--headless")
        cls.driver = webdriver.Chrome(options=options)
        cls.page = BooksPage(cls.driver)

    @classmethod
    def tearDownClass(cls):
        cls.driver.quit()
    
    def test_main_page_elements(self):
        """Проверка отображения основных элементов: логотип/название, навигационных ссылок, поиск."""
        self.page.load()
        # Проверяем наличие поля поиска
        search_box = self.driver.find_element(By.ID, "searchBox")
        self.assertIsNotNone(search_box, "Поле поиска должно присутствовать")
        # Проверяем наличие кнопки Login
        login_buttons = self.driver.find_elements(By.XPATH, "//button[text()='Login']")
        self.assertGreater(len(login_buttons), 0, "Кнопка 'Login' должна присутствовать")
        # Вместо поиска элемента по классу проверяем заголовок страницы
        self.assertIn("Book Store", self.driver.title, "Заголовок 'Book Store' не найден в title страницы")

    
    def test_main_menu(self):
        """Тестирование работы главного меню и подменю (если присутствует)."""
        self.page.load()
        # В demoqa.com/books меню может быть представлено шапкой страницы.
        try:
            menu = self.driver.find_element(By.XPATH, "//div[contains(@class, 'header-wrapper')]")
            self.assertIsNotNone(menu, "Главное меню должно присутствовать")
        except:
            self.skipTest("Главное меню отсутствует на данной странице")
    
    def test_quick_access_sections(self):
        """Проверка функционала быстрых ссылок (акции, новинки и т.д.)."""
        self.page.load()
        # Если быстрых ссылок нет, можно проверить наличие других ключевых элементов страницы.
        # Например, наличие блока с книгами.
        rows = self.page.get_book_rows()
        self.assertGreater(len(rows), 0, "Быстрый доступ: список книг не должен быть пустым")


In [4]:
# Ячейка 4: Тесты для страниц категорий и продуктов

class CategoriesProductsTests(unittest.TestCase):
    @classmethod
    def setUpClass(cls):
        options = webdriver.ChromeOptions()
        options.add_argument("--headless")
        cls.driver = webdriver.Chrome(options=options)
        cls.page = BooksPage(cls.driver)
    
    @classmethod
    def tearDownClass(cls):
        cls.driver.quit()
    
    def test_products_display(self):
        """Убедиться, что продукты (книги) отображаются корректно."""
        self.page.load()
        rows = self.page.get_book_rows()
        self.assertGreater(len(rows), 0, "Список книг не должен быть пустым")
    
    def test_filter_products(self):
        """Проверка работы фильтрации товаров через поисковую строку."""
        self.page.load()
        total_rows = len(self.page.get_book_rows())
        self.page.search_book("Git")
        filtered_rows = len(self.page.get_book_rows())
        self.assertLess(filtered_rows, total_rows, "Фильтрация должна уменьшать число отображаемых книг")
    
    def test_add_to_collection_button_presence(self):
        """Проверка наличия кнопки 'Add To Your Collection' на странице деталей книги."""
        self.page.load()
        self.page.search_book("Git Pocket Guide")
        clicked = self.page.click_book("Git Pocket Guide")
        self.assertTrue(clicked, "Не удалось кликнуть по книге 'Git Pocket Guide'")
        try:
            # Прокручиваем, чтобы элемент оказался в видимой области
            add_btn = WebDriverWait(self.driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//button[text()='Add To Your Collection']"))
            )
            self.driver.execute_script("arguments[0].scrollIntoView(true);", add_btn)
            self.assertIsNotNone(add_btn, "Кнопка 'Add To Your Collection' не найдена")
        except TimeoutException:
            self.fail("Кнопка 'Add To Your Collection' не появилась")


In [5]:
# Ячейка 5: Тесты для корзины/оформления заказа (работа с коллекцией книг)

class CartCheckoutTests(unittest.TestCase):
    # Используем тестовые учетные данные (предварительно зарегистрируйте пользователя)
    USERNAME = "TestUser"
    PASSWORD = "Test@1234"
    
    @classmethod
    def setUpClass(cls):
        options = webdriver.ChromeOptions()
        options.add_argument("--headless")
        cls.driver = webdriver.Chrome(options=options)
        cls.books_page = BooksPage(cls.driver)
        cls.login_page = LoginPage(cls.driver)
        cls.profile_page = ProfilePage(cls.driver)
        # Выполняем вход в систему
        cls.login_page.login(cls.USERNAME, cls.PASSWORD)
        # Дожидаемся перехода на профиль (или вручную переходим)
        WebDriverWait(cls.driver, 10).until(EC.url_contains("profile"))
    
    @classmethod
    def tearDownClass(cls):
        cls.driver.quit()
    
    def test_add_and_remove_book_from_collection(self):
        """Проверка добавления книги в коллекцию и последующего её удаления."""
        # Переходим на страницу книг
        self.books_page.load()
        # Ищем книгу
        self.books_page.search_book("Git Pocket Guide")
        clicked = self.books_page.click_book("Git Pocket Guide")
        self.assertTrue(clicked, "Не удалось перейти к деталям книги 'Git Pocket Guide'")
        # Добавляем книгу в коллекцию
        try:
            add_btn = WebDriverWait(self.driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//button[text()='Add To Your Collection']"))
            )
            add_btn.click()
            # Ждём и закрываем модальное окно (если появляется уведомление)
            time.sleep(1)
            try:
                WebDriverWait(self.driver, 5).until(EC.alert_is_present())
                alert = self.driver.switch_to.alert
                alert.accept()
            except NoAlertPresentException:
                pass
        except TimeoutException:
            self.fail("Кнопка 'Add To Your Collection' не стала кликабельной")
        
        # Переходим в профиль и проверяем наличие книги
        self.profile_page.load()
        collection = self.profile_page.get_books_in_collection()
        found = any("git pocket guide" in row.text.lower() for row in collection)
        self.assertTrue(found, "Книга не добавлена в коллекцию")
        
        # Удаляем книгу из коллекции
        removed = self.profile_page.remove_book("Git Pocket Guide")
        self.assertTrue(removed, "Не удалось удалить книгу из коллекции")


In [6]:
# Ячейка 6: Тесты для личного кабинета пользователя

class UserAccountTests(unittest.TestCase):
    # Тестовые данные для входа и регистрации
    USERNAME = "TestUser"      # Для регистрации можно генерировать уникальное имя
    PASSWORD = "Test@1234"
    
    @classmethod
    def setUpClass(cls):
        options = webdriver.ChromeOptions()
        options.add_argument("--headless")
        cls.driver = webdriver.Chrome(options=options)
        cls.login_page = LoginPage(cls.driver)
    
    @classmethod
    def tearDownClass(cls):
        cls.driver.quit()
    
    def test_login(self):
        """Проверка функционала входа в систему."""
        self.login_page.login(self.USERNAME, self.PASSWORD)
        # Ожидаем, что URL изменится на профиль
        WebDriverWait(self.driver, 10).until(EC.url_contains("profile"))
        self.assertIn("profile", self.driver.current_url, "После входа URL должен содержать 'profile'")
    
    def test_registration(self):
        """Проверка отображения формы регистрации."""
        self.driver.get("https://demoqa.com/register")
        try:
            reg_form = WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//input[@id='firstName' or @placeholder='First Name']"))
            )
            self.assertIsNotNone(reg_form, "Форма регистрации не найдена")
        except TimeoutException:
            self.skipTest("Форма регистрации не доступна на сайте")

    
    @unittest.skip("Редактирование личных данных не реализовано на данном сайте")
    def test_edit_personal_data(self):
        """Проверка функционала изменения личных данных."""
        pass
    
    @unittest.skip("История заказов отсутствует в демо-приложении")
    def test_order_history(self):
        """Проверка отображения истории заказов и их статусов."""
        pass


In [7]:
# Ячейка 7: Тесты для совместимости (проверка возможностей браузера)

class CompatibilityTests(unittest.TestCase):
    @classmethod
    def setUpClass(cls):
        options = webdriver.ChromeOptions()
        options.add_argument("--headless")
        cls.driver = webdriver.Chrome(options=options)
    
    @classmethod
    def tearDownClass(cls):
        cls.driver.quit()
    
    def test_browser_capability(self):
        """Проверка, что используется поддерживаемый браузер."""
        caps = self.driver.capabilities
        browser = caps.get("browserName", "").lower()
        self.assertIn(browser, ["chrome", "firefox", "safari", "edge"], "Неподдерживаемый браузер")
    
    def test_os_info(self):
        """Вывод информации об операционной системе (демонстрация совместимости)."""
        caps = self.driver.capabilities
        os_info = caps.get("platform", "Unknown")
        # Просто выводим информацию; тестируем здесь не будем
        print("OS Info:", os_info, file=sys.stderr)
        self.assertIsNotNone(os_info)


In [8]:
# Ячейка 8: Тесты производительности

class PerformanceTests(unittest.TestCase):
    @classmethod
    def setUpClass(cls):
        options = webdriver.ChromeOptions()
        options.add_argument("--headless")
        cls.driver = webdriver.Chrome(options=options)
        cls.page = BooksPage(cls.driver)
    
    @classmethod
    def tearDownClass(cls):
        cls.driver.quit()
    
    def test_page_load_time(self):
        """Оценка времени загрузки главной страницы."""
        start_time = time.time()
        self.page.load()
        load_time = time.time() - start_time
        print("Page load time:", load_time, "сек.", file=sys.stderr)
        # Устанавливаем порог в 5 секунд
        self.assertLess(load_time, 5, "Время загрузки страницы слишком велико")
    
    def test_response_under_slow_connection(self):
        """Демонстрация теста при имитации медленного соединения."""
        # Искусственно добавляем задержку и измеряем общее время загрузки
        start_time = time.time()
        self.page.load()
        time.sleep(2)  # имитируем задержку
        total_time = time.time() - start_time
        print("Total time under simulated slow connection:", total_time, "сек.", file=sys.stderr)
        self.assertLess(total_time, 10, "Слишком долгий отклик при медленном соединении")
    
    def test_interface_responsiveness(self):
        """Проверка отзывчивости интерфейса (время между кликом и отображением элемента)."""
        self.page.load()
        self.page.search_book("Git Pocket Guide")
        rows = self.page.get_book_rows()
        if rows:
            # Используем конкретную книгу для теста
            start_click = time.time()
            rows[0].click()
            try:
                add_btn = WebDriverWait(self.driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, "//button[text()='Add To Your Collection']"))
                )
                reaction_time = time.time() - start_click
                print("Interface reaction time:", reaction_time, "сек.", file=sys.stderr)
                self.assertLess(reaction_time, 5, "Интерфейс слишком медленно реагирует")
            except TimeoutException:
                self.fail("Элемент не появился после клика")
        else:
            self.fail("Нет доступных строк для клика")


In [9]:
# Ячейка 9: Тесты безопасности

class SecurityTests(unittest.TestCase):
    @classmethod
    def setUpClass(cls):
        options = webdriver.ChromeOptions()
        options.add_argument("--headless")
        cls.driver = webdriver.Chrome(options=options)
        cls.page = BooksPage(cls.driver)
    
    @classmethod
    def tearDownClass(cls):
        cls.driver.quit()
    
    def test_search_xss(self):
        """Проверка на XSS: ввод вредоносного скрипта не должен вызывать выполнение (alert не появляется)."""
        self.page.load()
        malicious_input = "<script>alert('XSS')</script>"
        self.page.search_book(malicious_input)
        time.sleep(1)
        try:
            alert = self.driver.switch_to.alert
            alert.dismiss()
            self.fail("Был вызван alert – возможна уязвимость XSS")
        except NoAlertPresentException:
            pass  # Тест пройден, alert не появился
    
    def test_search_injection(self):
        """Проверка устойчивости к SQL-инъекции."""
        self.page.load()
        injection_input = "' OR '1'='1"
        self.page.search_book(injection_input)
        time.sleep(1)
        rows = self.page.get_book_rows()
        self.assertIsInstance(rows, list, "Результат поиска должен быть списком даже после попытки инъекции")
    
    def test_ssl_certificate(self):
        """Проверка наличия SSL-сертификата (URL должен начинаться с https://)."""
        self.page.load()
        current_url = self.driver.current_url
        self.assertTrue(current_url.startswith("https://"), "URL должен использовать https")


In [10]:
# Ячейка 10: Тесты API для проверки базы данных

class DatabaseTests(unittest.TestCase):
    API_URL = "https://demoqa.com/BookStore/v1/Books"
    
    def test_api_status_code(self):
        """Проверка, что API возвращает статус 200."""
        response = requests.get(self.API_URL)
        self.assertEqual(response.status_code, 200, "API должен возвращать статус 200")
    
    def test_api_books_list(self):
        """Проверка, что ответ API содержит ключ 'books', являющийся списком."""
        response = requests.get(self.API_URL)
        data = response.json()
        self.assertIn("books", data, "Ответ должен содержать ключ 'books'")
        self.assertIsInstance(data["books"], list, "'books' должен быть списком")
    
    def test_api_book_fields(self):
        """Проверка, что у первой книги в списке присутствуют все необходимые поля."""
        response = requests.get(self.API_URL)
        data = response.json()
        if data["books"]:
            book = data["books"][0]
            expected_fields = ["isbn", "title", "subTitle", "author", "publish_date", 
                               "publisher", "pages", "description", "website"]
            for field in expected_fields:
                self.assertIn(field, book, f"У книги должно быть поле '{field}'")


In [11]:
# Ячейка 11: Запуск всех тестов с выводом сводной таблицы результатов

import sys

# Кастомный класс для сбора результатов тестов
class CustomTestResult(unittest.TextTestResult):
    def __init__(self, stream, descriptions, verbosity):
        super().__init__(stream, descriptions, verbosity)
        self.test_details = []  # список для хранения деталей теста
        self.test_number = 1

    def addSuccess(self, test):
        super().addSuccess(test)
        desc = test.shortDescription() or str(test)
        self.test_details.append((self.test_number, desc, "OK"))
        self.test_number += 1

    def addFailure(self, test, err):
        super().addFailure(test, err)
        desc = test.shortDescription() or str(test)
        self.test_details.append((self.test_number, desc, "FAIL"))
        self.test_number += 1

    def addError(self, test, err):
        super().addError(test, err)
        desc = test.shortDescription() or str(test)
        self.test_details.append((self.test_number, desc, "ERROR"))
        self.test_number += 1

    def addSkip(self, test, reason):
        super().addSkip(test, reason)
        desc = test.shortDescription() or str(test)
        self.test_details.append((self.test_number, desc, f"SKIPPED ({reason})"))
        self.test_number += 1

# Кастомный раннер, использующий наш класс результатов
class CustomTextTestRunner(unittest.TextTestRunner):
    resultclass = CustomTestResult

if __name__ == '__main__':
    # Собираем все тесты из текущего модуля
    suite = unittest.TestLoader().loadTestsFromModule(sys.modules[__name__])
    # Запускаем тесты с использованием кастомного раннера
    runner = CustomTextTestRunner(verbosity=2)
    result = runner.run(suite)
    
    # Вывод сводной таблицы с результатами тестов
    print("\nSummary Table:")
    header = "{:<5} {:<70} {:<15}".format("№", "Test Description", "Result")
    print(header)
    print("-" * len(header))
    for num, desc, outcome in result.test_details:
         print("{:<5} {:<70} {:<15}".format(num, desc, outcome))


test_add_and_remove_book_from_collection (__main__.CartCheckoutTests.test_add_and_remove_book_from_collection)
Проверка добавления книги в коллекцию и последующего её удаления. ... FAIL
test_add_to_collection_button_presence (__main__.CategoriesProductsTests.test_add_to_collection_button_presence)
Проверка наличия кнопки 'Add To Your Collection' на странице деталей книги. ... FAIL
test_filter_products (__main__.CategoriesProductsTests.test_filter_products)
Проверка работы фильтрации товаров через поисковую строку. ... FAIL
test_products_display (__main__.CategoriesProductsTests.test_products_display)
Убедиться, что продукты (книги) отображаются корректно. ... ok
test_browser_capability (__main__.CompatibilityTests.test_browser_capability)
Проверка, что используется поддерживаемый браузер. ... ok
test_os_info (__main__.CompatibilityTests.test_os_info)
Вывод информации об операционной системе (демонстрация совместимости). ... OS Info: Unknown
ok
test_api_book_fields (__main__.DatabaseTes


Summary Table:
№     Test Description                                                       Result         
--------------------------------------------------------------------------------------------
1     Проверка добавления книги в коллекцию и последующего её удаления.      FAIL           
2     Проверка наличия кнопки 'Add To Your Collection' на странице деталей книги. FAIL           
3     Проверка работы фильтрации товаров через поисковую строку.             FAIL           
4     Убедиться, что продукты (книги) отображаются корректно.                OK             
5     Проверка, что используется поддерживаемый браузер.                     OK             
6     Вывод информации об операционной системе (демонстрация совместимости). OK             
7     Проверка, что у первой книги в списке присутствуют все необходимые поля. OK             
8     Проверка, что ответ API содержит ключ 'books', являющийся списком.     OK             
9     Проверка, что API возвращает статус 200. 